# Statistic-Based Neural Network for Normality Testing

## Set up the environment

In [ ]:
# Import everything that's needed to run the notebook
import os
import pickle
import pathlib
import datetime
import random

from IPython.display import display, Markdown, Latex
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import sklearn.metrics
import sklearn.preprocessing
import scipy.stats
import matplotlib.pyplot as plt
import boruta

import util

Read the configuration.

In [ ]:
# Define the path to the configuration dictionary
config_path = 'configuration.p'

# Load the configuration dictionary
with open(config_path, 'rb') as f:
    configuration = pickle.load(f)
    
# Get the paths to the relevant directories 
data_directory_path = configuration['data']['directory_path']
classifiers_directory_path = configuration['classifiers']['directory_path']

# Get the parameters of the experiment
cv_folds = configuration['experiment']['number_of_cv_folds']

## Prepare the data

### Load the data
Load the datasets using the function `load_from_file` from `util`.

In [ ]:
# Define the dictionary to store the actual datasets, indexed by their names
datasets = {}

# Load the datasets
for set_name in ['A']:
    set_path = configuration['data']['datasets'][set_name]['path']
    print('Loading {} from {}'.format(set_name, set_path))
    datasets[set_name] = util.load_from_file(set_path)
    print('Done.')

### Split $\mathcal{A}$ into Cross-validation and Test  Subsets ($\mathcal{A}_{cv}$ and $\mathcal{A}_{test}$) 

Split $\mathcal{A}$ into the subsets for cross-validation ($\mathcal{A}_{cv})$ and testing ($\mathcal{A}_{test}$).

Use 70% of the set to cross-validate and 30% for subsequent testing and evaluation.

In [ ]:
# Extract the labels from the set, leaving only samples in it
labels = [labeled_sample.pop() for labeled_sample in datasets['A']]
samples = datasets['A']

# There is no need to store the sama data twice, in datasets['A'] and in (samples, labels)
del datasets['A']

# Define the stratification labels as the combination of actual labels and sample sizes
stratification_labels = [str(label) + str(len(sample)) for (label, sample) in zip(labels, samples)]

# Set the relative size of the CV subset
train_size = 0.7

# Split the data into CV and test subsets
set_A_cv, set_A_test, y_cv, y_test = train_test_split(samples, labels, stratify=stratification_labels, 
                                              train_size=train_size)

## Construct the Statistic-based Feedforward Neural Network Classifier (SBNN)

### Prepare the Preprocessor

In [ ]:
def lin_mudholkar_statistic(sample, tol=1e-7):
    n = len(sample)
    sum_of_squares = 0
    for x in sample:
        sum_of_squares = sum_of_squares + x**2
        
    h_values = [0 for i in range(n)]
    for i in range(n):
        x = sample[i]
        
        corrected_sum = 0
        for j in range(n):
            if j != i:
                corrected_sum = corrected_sum + sample[j]
                
        square_of_sum = corrected_sum**2
        difference = (((sum_of_squares - x**2) - square_of_sum / (n - 1)) / n)
        if abs(difference) <= tol:
            difference = 0
        h_i = difference**(1/3)
        h_values[i] = h_i
        
        #print(i, x, corrected_sum, square_of_sum, square_of_sum / (n - 1), '\n', h_i)
    
    r = np.corrcoef(sample, h_values)
    
    return np.arctan(r[0, 1])

def vasicek_statistic(sample, m=3):
    n = len(sample)
    sample = np.array(sample, dtype=np.float64)
    sd = np.std(sample)
    sorted_sample = np.sort(sample)
    product = 1
    m = m
    for i in range(n):
        if i - m < 0:
            left = sorted_sample[0]
        else:
            left = sorted_sample[i - m]
        
        if i + m > n - 1:
            right = sorted_sample[n - 1]
        else:
            right = sorted_sample[i + m]
        
        product = product * (right - left)
    
    return (n / (2 * m * sd)) * (product ** (1 / n))

In [ ]:
def preprocess(sample):
    n = len(sample)
    skewness = scipy.stats.skew(sample)
    kurtosis = scipy.stats.kurtosis(sample, fisher=False)
    W = scipy.stats.shapiro(sample).statistic
    lm_stat = lin_mudholkar_statistic(sample)
    K3 = vasicek_statistic(sample, m=3)
    K5 = vasicek_statistic(sample, m=5)
    return [skewness, kurtosis, W, lm_stat, K3, K5, n]
    

In [ ]:
class SBNNPreprocessor(TransformerMixin, BaseEstimator):
    def __init__(self):
        super(SBNNPreprocessor, self).__init__()
        
        # Set the names of the features in the descriptors
        self.features = ['skewness', 'kurtosis', 'W', 'LN-statistic', 'K3', 'K5', 'n']
        
    def fit(self, X, y=None):
        # Not needed, but present for compatibility.
        return self
    
    def transform(self, X, y=None):
        # Note: Currently working only on a list of lists or a single list.
        if isinstance(X, list):
            if all(isinstance(x, list) for x in X):
                X = [preprocess(x) for x in X]
                return pd.DataFrame(X)
            else:
                X = preprocess(X)
                return X
        else:
            # Pandas dataframes and numpy arrays are not supported for now.
            pass

### Cross-validate
Create a `sklearn` pipeline that consists of the preprocessor, standard scaler, mean imputer to replace the null values, and the neural network itself.

In [ ]:
preprocessor = SBNNPreprocessor()
scaler = sklearn.preprocessing.StandardScaler()
imputer = SimpleImputer(strategy='mean')
neural_net = MLPClassifier(solver='adam', max_iter=200, activation='relu',
                           early_stopping=True, validation_fraction=0.1)
pipe = Pipeline([('preprocessor', preprocessor),
                 ('scaler', scaler),
                 ('imputer', imputer),
                 ('neural_net', neural_net),
                ])

Run the grid search to fit the network's parameters and find the best hyperparameters.

In [ ]:
# Specify the hyperparameter grid
param_grid = dict(
                  neural_net__hidden_layer_sizes = [(1000), (100, 10)],
                  neural_net__alpha = [10, 1, 0.1],
                 )

# Define the grid search object
grid = GridSearchCV(pipe,
                    param_grid=param_grid,
                    scoring='accuracy',
                    refit=True,
                    cv=cv_folds,
                    verbose=1,
                    n_jobs=-1)

# Perform cross-validation
grid.fit(set_A_cv, y_cv)

In [ ]:
# Get the trained network.
sbnn = grid.best_estimator_

# Show its hyperparameters
grid.best_params_

In [ ]:
# Show the means and deviations of the score(s) and CV time
params = grid.cv_results_['params']
mean_scores = grid.cv_results_['mean_test_score']
score_sds = grid.cv_results_['std_test_score']
mean_fit_times = grid.cv_results_['mean_fit_time']
time_sds = grid.cv_results_['std_fit_time']

results = []
for (params, mean_score, score_sd, mean_fit_time, time_sd) in zip(params, mean_scores, score_sds, mean_fit_times, time_sds):
    alpha = params['neural_net__alpha']
    structure = params['neural_net__hidden_layer_sizes']
    results.append([str(structure), alpha, mean_score, score_sd, mean_fit_time, time_sd])

results_df = pd.DataFrame(results)
results_df.columns = ['structure', 'c', 'mean_score', 'score_sd', 'mean_time', 'time_sd']
results_df

## Save the SBNN

In [ ]:
with open(classifiers_directory_path + '/sbnn.p', 'wb') as f:
    pickle.dump(sbnn, f)